# Cryptocurrency Analysis with Python: Retrieving Crypto Data from CoinGecko
Find more tutorials about Computer Vision, Microscopy, Biology and Data Science [here](https://medium.com/@microbioscopicdata)

Welcome back to our tutorial series on Cryptocurrency Analysis with Python! In our previous tutorials, we explored how to use powerful Python libraries such as Matplotlib, mplfinance, and yfinance to load and visualize cryptocurrency data from popular sources like Yahoo Finance. We’ve discussed about cryptocurrency exchanges, trade expenses (visible and hidden costs) and what you should really know about them before you create an account and before you start trading with them. We’ve also introduced various impactful risk-adjusted performance metrics: such as the Calmar Ratio, the Sharpe Ratio, and the Sortino Ratio. We successfully implemented the Simple Moving Average (SMA) Crossover Strategy, calculated associated trading costs encompassing commissions, spreads, and slippage, and further optimized the strategy by exploring diverse parameter combinations to identify optimal settings.

In this  tutorial, we will delve into the process of retrieving detailed and in-depth cryptocurrency data from CoinGecko, a popular data aggregator, using Python.
    

Disclaimer: This article is not financial advice. This is purely introductory knowledge. All investment-related queries should be directed to your financial advisor.

Disclaimer: I am not affiliated with CoinGecko. I personally utilize their platform for retrieving cryptocurrency data.

## Cryptocurrency data aggregators

Cryptocurrency data aggregators simplify the process of accessing a wide range of brokers and cryptocurrency assets through a single interface. They do this by utilizing Application Programming Interfaces (APIs), which act as bridges for different computer programs to communicate and get prices and diverse market data from various exchanges. This information is then presented in a unified format, making it user-friendly. These aggregators also offer additional features such as charts, portfolio tracking, live news updates, and customizable alerts [1].

In this tutorial, we'll be using CoinGecko, a popular data aggregator, for several reasons:

- It consolidates data from numerous cryptocurrency exchanges and sources, providing a comprehensive view of the cryptocurrency market.
- It covers a wide array of cryptocurrencies, not just the most popular ones.
- It offers historical data, allowing us to analyze price trends, trading volumes, and market capitalization over time.
- It provides various market metrics, including trading volumes, market capitalization, circulating supply, and more.
- Most importantly, it offers a free and powerful API that enables us to retrieve real-time cryptocurrency data efficiently.


One of the limitations of CoinGecko is in its free version, which imposes a rate limit of 20-30 calls per minute (meaning that we make a maximum of 20 to 30 API requests/calls to their server in a single minute). However, we can address this issue by implementing Python code and incorporating waiting times [2].

CoinGecko, there are several other cryptocurrency data aggregators and sources available. Here are some of them:

- CoinMarketCap
- Messari 
- CoinCap
- Coinlore  

and many more...

## Using CoinGecko API 

In order to interact with CoinGecko's data and services, we'll use the CoinGecko API wrapper [**pycoingecko**](https://github.com/man-c/pycoingecko). To install the pycoingecko, we must use `pip install -U pycoingecko` command you in our computer's command prompt or terminal. Alternatively, we can use the `!pip install -U pycoingecko` command directly in our Jupyter Notebook [3].


The code below imports the Python libraries we will use in this tutorial, creates an instance of the CoinGeckoAPI class, assigns it to the variable `cg`, and checks the connectivity to the server.

In [1]:
# !pip install -U pycoingecko # install the CoinGecko API wrapper, pycoingecko

# Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
from pycoingecko import CoinGeckoAPI
import time
import math


# Create an instance of the CoinGeckoAPI class and assigns it to the variable cg
cg = CoinGeckoAPI() # connect

# Check API connectivity using ping method
cg.ping()

{'gecko_says': '(V3) To the Moon!'}

### Simple calls for one coin

The code below retrieves the current price of Bitcoin (BTC) in Euros (EUR). The parameter:

- `ids` specifies the cryptocurrency we want to retrieve the price for.
- `vs_currencies` specifies the fiat currency in which we want to get the price. Here, we select the euro, but we can pass also other currencies. We can retrieve all the supported fiat currencies using  `cg.get_supported_vs_currencies()` function.

In [2]:
# Retrieve data for Bitcoin (BTC) from the CoinGecko API
cg.get_price(ids="bitcoin", vs_currencies ="eur")

{'bitcoin': {'eur': 24300}}

### Simple calls for one coin with more parameters

The code below retrieves the current price of Bitcoin (BTC) in Euros (EUR,) also inludes its market capitalization, 24-hour trading volume, 24-hour price change, and the timestamp of the last update. 

In [3]:
# Retrieve data for Bitcoin (BTC) from the CoinGecko API
data = cg.get_price(ids="bitcoin", vs_currencies ="eur", include_market_cap=True,
            include_24hr_vol=True, include_24hr_change=True, include_last_updated_at=True)

# Convert the retrieved data (in JSON format) into a pandas DataFrame
data = pd.DataFrame(data)

# Convert the "last_updated_at" column of the Bitcoin data to a datetime format.
data["bitcoin"]["last_updated_at"] = pd.to_datetime(data["bitcoin"]["last_updated_at"], unit="s")
data

,bitcoin
eur,24307.0
eur_24h_change,1.278685
eur_24h_vol,13509952757.84778
eur_market_cap,473235431182.347534
last_updated_at,2023-08-23 16:59:59


### Simple calls for more coins with more parameters

If we want to download data for more cryptocurrencies, we can pass a list of the cryptocurrency symbols* we want to retrieve prices for in our code above. Additionally, we'll iterate over each column to convert the 'last_updated_at' column for each cryptocurrency to a datetime format.

**For retrieving the symbol of a cryptocurrency, we can search for it on the CoinGecko page.*

In [5]:
# Retrieve data for Bitcoin (BTC) from the CoinGecko API
data = cg.get_price(ids=["bitcoin","ethereum","binancecoin","polkadot"], vs_currencies ="eur", include_market_cap=True,
            include_24hr_vol=True, include_24hr_change=True, include_last_updated_at=True)

# Convert the retrieved data (in JSON format) into a pandas DataFrame
data = pd.DataFrame(data)

# Iterate through the columns and convert "last_updated_at" to datetime
for column in data.columns:
    # Convert the "last_updated_at" column of every crypto  to a datetime format.
    data[column]["last_updated_at"] = pd.to_datetime(data[column]["last_updated_at"], unit="s")
data

C:\Users\Alexandros\AppData\Local\Temp\ipykernel_20868\941445899.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column]["last_updated_at"] = pd.to_datetime(data[column]["last_updated_at"], unit="s")


,binancecoin,bitcoin,ethereum,polkadot
eur,200.1,24309.0,1548.59,4.14
eur_market_cap,30799643945.131981,472891670928.997986,186036235412.075165,5251980757.137348
eur_24h_vol,622495061.55755,10086202841.38879,11852883996.348318,106214732.109125
eur_24h_change,4.935331,1.432255,2.106671,2.938505
last_updated_at,2023-08-23 17:08:34,2023-08-23 17:08:35,2023-08-23 17:08:31,2023-08-23 17:08:29


## Retrieve data for all available coins in CoinGecko

### Retrieve the list with all available coins in CoinGecko

To retrieve all available coins in CoinGecko, we'll use the function `cg.get_coins_list()`.

In [6]:
# Retrieve a list of available cryptocurrencies from the CoinGecko API
crypto_list = cg.get_coins_list()

# Convert the list into a pandas DataFrame for easier manipulation and analysis
crypto_list = pd.DataFrame(crypto_list)

### Find the duplicated names and symbols
However, we must be careful because not all names or symbols are unique, as we can see below when using the `nunique()` function. The number of coins is calculated by the different IDs.

In [7]:
# Use the nunique() function to count the number of unique values in each column
unique_counts = crypto_list.nunique()
unique_counts

id        10150
symbol     8569
name      10132
dtype: int64

In order to find the duplicated names and symbols, we are going to use the code below that includes the `.duplicated` function.

In [8]:
# Check for duplicate cryptocurrency names
duplicate_names = crypto_list[crypto_list['name'].duplicated(keep=False)]

# Check for duplicate cryptocurrency symbols
duplicate_symbols = crypto_list[crypto_list['symbol'].duplicated(keep=False)]

# Display the duplicate names and symbols
print("Duplicate Names:")
print(duplicate_names[["id",'name', 'symbol']])

print("\nDuplicate Symbols:")
print(duplicate_symbols[["id",'name', 'symbol']])

Duplicate Names:
                       id            name        symbol
388         alpha-finance          Stella         alpha
863        baby-shiba-inu  Baby Shiba Inu  babyshibainu
864    baby-shiba-inu-erc  Baby Shiba Inu      babyshib
989                  bean            Bean          bean
990                bean-2            Bean          bean
2870                 dyor            DYOR          dyor
2872         dyor-token-2            DYOR          dyor
2994                ember           Ember         ember
2995              ember-2           Ember          embr
3222            ezzy-game       EZZY Game           ezy
3223          ezzy-game-2       EZZY Game          gezy
4178                homer           Homer       simpson
4179              homer-2           Homer   simpson 2.0
4867                 kuku            KuKu          kuku
4868             kuku-eth            KuKu          kuku
5541         meta_finance    Meta Finance           mf1
5542         meta-finance    Me

To retrieve market data (ordered by market capitalization in descending order) for all available coins in CoinGecko, we'll use the function `cg.get_coins_markets()`.   

**For an unknown reason, I wasn't able to set the `per_page` parameter to its maximum value of 250 (the code did not run). For this reason, I left it at the default value of 100.**


In [9]:
data = cg.get_coins_markets(vs_currency ="eur",page=1, order="market_cap_dec")
df = pd.DataFrame(data)
df

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,24317.000000,472891670929,1,5.101599e+11,1.047091e+10,24326.000000,...,2.100000e+07,2.100000e+07,59717.00,-59.29559,2021-11-10T14:24:11.849Z,5.130000e+01,47284.28614,2013-07-05T00:00:00.000Z,None,2023-08-23T17:09:06.725Z
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,1549.100000,186036235412,2,1.860362e+11,1.188472e+10,1548.170000,...,1.202143e+08,NaN,4228.93,-63.40126,2021-12-01T08:38:24.623Z,3.814550e-01,405644.76869,2015-10-20T00:00:00.000Z,"{'times': 84.18914544508462, 'currency': 'btc'...",2023-08-23T17:09:02.594Z
2,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,0.920067,76219901934,3,7.621990e+10,1.198531e+10,0.926721,...,8.284578e+10,NaN,1.13,-18.69726,2018-07-24T00:00:00.000Z,5.330960e-01,72.58097,2015-03-02T00:00:00.000Z,None,2023-08-23T17:05:00.377Z
3,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,200.190000,30799643945,4,4.003694e+10,6.212806e+08,200.600000,...,1.538562e+08,2.000000e+08,583.17,-65.66038,2021-11-26T02:58:28.000Z,3.359941e-02,595921.50554,2017-10-19T00:00:00.000Z,None,2023-08-23T17:09:05.318Z
4,ripple,xrp,XRP,https://assets.coingecko.com/coins/images/44/l...,0.488026,25738616573,5,4.870876e+10,9.735330e+08,0.487065,...,9.998850e+10,1.000000e+11,2.82,-82.74971,2018-01-07T00:00:00.000Z,1.946190e-03,24922.21376,2013-08-16T00:00:00.000Z,None,2023-08-23T17:09:01.287Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,mina-protocol,mina,Mina Protocol,https://assets.coingecko.com/coins/images/1562...,0.375513,359128299,96,4.006886e+08,6.206394e+06,0.375969,...,1.066376e+09,NaN,7.43,-94.94754,2021-06-01T01:42:37.064Z,3.424600e-01,9.63998,2023-08-17T21:49:55.138Z,None,2023-08-23T17:09:02.548Z
96,huobi-token,ht,Huobi,https://assets.coingecko.com/coins/images/2822...,2.210000,352531551,97,4.446284e+08,1.203294e+07,2.220000,...,2.010500e+08,2.010500e+08,32.84,-93.26375,2021-05-12T14:42:21.586Z,2.744040e-01,706.12657,2023-03-10T05:05:00+08:00,"{'times': 1.4278784466311434, 'currency': 'usd...",2023-08-23T17:09:05.603Z
97,terra-luna,lunc,Terra Luna Classic,https://assets.coingecko.com/coins/images/8284...,0.000060,348794454,98,4.101944e+08,1.731924e+07,0.000060,...,6.838169e+12,NaN,108.71,-99.99994,2022-04-05T12:24:58.854Z,9.627430e-07,6130.50789,2022-05-13T02:34:40.340Z,None,2023-08-23T17:09:00.865Z
98,dydx,dydx,dYdX,https://assets.coingecko.com/coins/images/1750...,1.970000,339334347,99,1.959879e+09,7.161093e+07,1.960000,...,1.000000e+09,1.000000e+09,24.01,-91.82204,2021-09-30T01:23:43.570Z,9.535960e-01,105.89459,2022-12-30T12:30:10.546Z,None,2023-08-23T17:09:01.775Z


The limitation of the code above is that **we are retrieving data for the top 100 cryptocurrencies**. To retrieve market data for all available cryptocurrencies using the CoinGecko API, you typically need to paginate through the data, as there are too many cryptocurrencies to retrieve in a single request (we have found previously more than 10000 available coins using the `cg.get_coins_list()` function). 


In [13]:
# Define an empty DataFrame to store the results
all_crypto_data = pd.DataFrame()

# Set the number of cryptocurrencies to retrieve per page and initialize the page number
page = 1

while True:
    # Retrieve market data for the current page of cryptocurrencies
    data = cg.get_coins_markets(vs_currency="eur", order="market_cap_desc", page=page)

    # Check if the data is empty, indicating no more results
    if not data:
        print("Download completed")
        break

    # Convert the current page of data into a DataFrame
    page_crypto_data = pd.DataFrame(data)

    # Append the current page's data to the overall DataFrame
    all_crypto_data = pd.concat([all_crypto_data, page_crypto_data], ignore_index=True)

    # Increment the page number for the next request
    page += 1
    print(f"Page: {page}", end="\r")
    # keep in mind that the CoinGecko  have rate limits when fetching large datasets, if needed, use time.sleep
    time.sleep(1)

# Now, all_crypto_data contains market data for all available cryptocurrencies
all_crypto_data

Download completed


,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,24364.000000,474520923430.0,1.0,5.119166e+11,1.624614e+10,24433.0,...,2.100000e+07,2.100000e+07,59717.0,-59.18370,2021-11-10T14:24:11.849Z,51.3,47414.53965,2013-07-05T00:00:00.000Z,None,2023-08-23T18:42:47.046Z
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,1546.270000,185987617154.0,2.0,1.859876e+11,1.187268e+10,1553.38,...,1.202143e+08,NaN,4228.93,-63.42011,2021-12-01T08:38:24.623Z,0.381455,405435.72792,2015-10-20T00:00:00.000Z,"{'times': 83.84405195701511, 'currency': 'btc'...",2023-08-23T18:42:50.617Z
2,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,0.920031,76220647448.0,3.0,7.622065e+10,1.414680e+10,0.926721,...,8.284578e+10,NaN,1.13,-18.69647,2018-07-24T00:00:00.000Z,0.533096,72.58266,2015-03-02T00:00:00.000Z,None,2023-08-23T18:40:00.329Z
3,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,199.990000,30778531852.0,4.0,4.000949e+10,6.612592e+08,200.63,...,1.538562e+08,2.000000e+08,583.17,-65.70086,2021-11-26T02:58:28.000Z,0.033599,595218.98232,2017-10-19T00:00:00.000Z,None,2023-08-23T18:42:50.796Z
4,ripple,xrp,XRP,https://assets.coingecko.com/coins/images/44/l...,0.486917,25770400977.0,5.0,4.870225e+10,9.332927e+08,0.491048,...,9.998849e+10,1.000000e+11,2.82,-82.74995,2018-01-07T00:00:00.000Z,0.001946,24921.86548,2013-08-16T00:00:00.000Z,None,2023-08-23T18:42:47.909Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10145,cola-token-2,cola,Cola Token,https://assets.coingecko.com/coins/images/3093...,NaN,None,None,NaN,NaN,None,...,NaN,NaN,None,0.00000,None,None,0.00000,None,None,None
10146,fiero,fiero,Fiero,https://assets.coingecko.com/coins/images/3036...,NaN,None,None,NaN,NaN,None,...,NaN,NaN,None,0.00000,None,None,0.00000,None,None,None
10147,amplifi-dao,agg,AmpliFi DAO,https://assets.coingecko.com/coins/images/2913...,NaN,None,None,NaN,NaN,None,...,3.000000e+09,NaN,None,0.00000,None,None,0.00000,None,None,None
10148,ayin,ayin,Ayin,https://assets.coingecko.com/coins/images/3131...,NaN,None,None,NaN,NaN,None,...,1.924590e+05,4.000000e+06,None,0.00000,None,None,0.00000,None,None,None


In [14]:
# Save the DataFrame to a CSV file
all_crypto_data.to_csv('all_crypto_data.csv', index=False)

## Conclusions

In this tutorial, we explored how to retrieve market data for all available cryptocurrencies from a popular data aggregator, CoinGecko using Python. We saw how to utilize the CoinGecko API wrapper [**pycoingecko**](https://github.com/man-c/pycoingecko) to overcome the limitations of retrieving large datasets (market data for all available cryptocurrencies), due to limitations of free use on CoinGecko. With the obtained dataset, we have the flexibility to perform various analyses, track market trends, and make informed decisions. Before that, however, we need to understand and clean the data, which we will explore in our next tutorial.




If you enjoy reading stories of this nature and wish to show your support for my writing, you may contemplate becoming a Medium member. By subscribing for just $5 per month, you’ll gain boundless entry to a vast collection of Python guides and Data science articles. Additionally, if you choose to sign up through my referral link, I’ll receive a modest commission at no extra cost to you.

## References:
[1]	“Top 15 Cryptocurrency Data Aggregators that Everyone Should Use – Cryptopolitan.” https://www.cryptopolitan.com/top-15-cryptocurrency-data-aggregators/ (accessed Aug. 23, 2023).  

[2]	“Crypto API Pricing Plans,” CoinGecko. https://www.coingecko.com/en/api/pricing (accessed Aug. 23, 2023).  

[3]	M. Christoforou, “CoinGecko API wrapper.” Aug. 06, 2023. Accessed: Aug. 23, 2023. [Online]. Available: https://github.com/man-c/pycoingecko
